In [1]:
from edgar_functions_V2 import *
import numpy as np

ticker = "AMD"

In [2]:
scorecard_order = [
    "Net Revenue",
    "Cost of Revenue",
    "Gross Profit",
    "Gross Margin %",
    "SG&A",
    "Operating Income",
    "Operating Margin %",
    "Interest Expense",
    "EBT",
    "Tax Expense",
    "Tax Rate %",
    "Net Income",
    "EPS",
    "EPS Diluted",
    "EBITDA",
    "EBITDA Margin %",
    "Dividend",
    "Dividend Payout Ratio %",
    "Cash",
    "Marketable Securities",
    "Inventory",
    "Accounts Receivable",
    "Other Current Assets",
    "Total Current Assets",
    "Property, Plant, and Eqipment",
    "Right of Use Assets",
    "Other Long Term Assets",
    "Goodwill",
    "Intangible Assets",
    "Total Assets",
    "Accounts Payable",
    "Accrued Expenses",
    "Deferred Revenue",
    "Income Tax Payable",
    "Operating Lease Liability",
    "Other Current Liabilites",
    "Total Current Liabilities",
    "Long Term Debt",
    "Other Long Term Liabilities",
    "Total Liabilities",
    "Additional Paid In Capital",
    "Retained Earnings",
    "Total Stockholders Equity",
    "Total Liabilities and Stockholders Equity",
    "Shares Outstanding (Diluted)",
    "Book Value Per Share",
    "Operating Cash Flow",
    "OCF/NI",
    "Depreciation and Amortization",
    "Capex",
    "Capex / Depreciation",
    "Free Cash Flow",
    "Dividend Payment",
    "Stock Repurchase",
    "ROA %",
    "ROE %",
    "Profit Margin %",
    "Asset Turnover",
    "Current Ratio",
    "Equity Multiplier",
    "Net Working Capital",
    "Debt to Equity Ratio",
    "Debt to Assets Ratio",
    "Days Sales Outstanding",
    "Days of Inventory on Hand",
    "Payables Period",
    "Receivables Turnover",
    "Cash Conversion Cycle",
]

In [3]:
def combine_or_add_columns(df, cols, add_instead=False, replace_zero=False):
    if not cols:
        return None

    primary_col = None

    for col in cols:
        if col in df.columns:
            primary_col = df[col]
            break

    if primary_col is None:
        return None

    if len(cols) == 1:
        return primary_col

    for col in cols[1:]:
        if col in df.columns:
            # Skip if the column contains all NaN values
            if df[col].isna().all():
                continue

            if add_instead:
                primary_col = primary_col.add(df[col], fill_value=0).combine_first(
                    primary_col.combine_first(df[col])
                )
            else:
                if replace_zero:
                    primary_col = primary_col.mask(primary_col == 0).combine_first(
                        df[col]
                    )
                else:
                    primary_col = primary_col.combine_first(df[col])

    return primary_col

In [4]:
annual_df = process_and_aggregate_annual_data(ticker)
income_mapping = income_statement_mapping.income_mapping
balance_mapping = balance_sheet_mapping.balance_mapping
cash_mapping = cash_flow_mapping.cash_mapping
income = get_changed_columns(income_mapping, annual_df).droplevel(1, axis=1)
balance = get_changed_columns(balance_mapping, annual_df).droplevel(1, axis=1)
cash_flow = get_changed_columns(cash_mapping, annual_df).droplevel(1, axis=1)

income_df = income.T.groupby(income.columns).mean().sort_index(ascending=False)
income_df = income_df.sort_index(axis=1, ascending=False)
income_df = income_df.T
balance_df = balance.T.groupby(balance.columns).mean().sort_index(ascending=False)
balance_df = balance_df.T.sort_index(axis=1, ascending=False)
cash_df = cash_flow.T.groupby(cash_flow.columns).mean().sort_index(ascending=False)
cash_df = cash_df.T.sort_index(axis=1, ascending=False)

In [5]:
# for income statement
if "Cost of Revenue" not in income_df.columns:
    income_df["Cost of Revenue"] = income_df["Net Revenue"] - income_df["Gross Profit"]
income_df["EBITDA"] = (
    income_df["EBT"]
    + income_df["Interest Expense"].notna()
    + income_df["Depreciation and Amortization"]
)
if "Tax Rate Continuing Operations" not in income_df.columns:
    income_df["Tax Rate Continuing Operations"] = -income_df["Taxes"] / income_df["EBT"]
# for balance sheet
if "Total Liabilities" not in balance_df.columns:
    balance_df["Total Liabilities"] = (
        balance_df["Total Liabilities and Stockholders Equity"]
        - balance_df["Total Stockholders Equity"]
    )

if "Total Stockholders Equity" not in balance_df.columns:
    balance_df["Total Stockholders Equity"] = (
        balance_df["Total Assets"] - balance_df["Total Liabilities"]
    )
# for cash flow
cash_df["Depreciation and Amortization(calc)"] = combine_or_add_columns(
    cash_df, ["Depreciation", "Amortization"], add_instead=True
)

In [6]:
annual_df.T.sort_index(axis=1, ascending=False)

Year,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
fact,,,,,,,,,,,,,,,
Period End,2022-12-31 00:00:00,2021-12-25 00:00:00,2020-12-26 00:00:00,2019-12-28 00:00:00,2018-12-29 00:00:00,2017-12-30 00:00:00,2016-12-31 00:00:00,2015-12-26 00:00:00,2014-12-27 00:00:00,2013-12-28 00:00:00,2012-12-29 00:00:00,2011-12-31 00:00:00,2010-12-25 00:00:00,2009-12-26 00:00:00,2008-12-27 00:00:00
Filing Date,2023-02-27 00:00:00,2022-02-03 00:00:00,2021-01-29 00:00:00,2020-02-04 00:00:00,2019-02-08 00:00:00,2018-02-27 00:00:00,2017-02-21 00:00:00,2016-02-18 00:00:00,2015-02-19 00:00:00,2014-02-17 00:00:00,2013-02-16 00:00:00,2012-02-16 00:00:00,2011-02-15 00:00:00,2010-02-13 00:00:00,2009-02-12 00:00:00
AccountsPayableCurrent,"2,493,000,000","1,321,000,000","468,000,000","988,000,000","834,000,000","384,000,000","440,000,000","279,000,000","415,000,000","519,000,000","278,000,000","363,000,000","376,000,000","647,000,000",NaN
AccountsPayableRelatedPartiesCurrent,"463,000,000","85,000,000","78,000,000","213,000,000","207,000,000","412,000,000","383,000,000",0,"218,000,000","364,000,000","454,000,000","177,000,000","205,000,000",NaN,NaN
AccountsReceivableGrossCurrent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"832,000,000","632,000,000","921,000,000","972,000,000",NaN,NaN
AccountsReceivableNetCurrent,"4,126,000,000","2,706,000,000","2,066,000,000","1,859,000,000","1,235,000,000","454,000,000","372,000,000","533,000,000","818,000,000","832,000,000","630,000,000","919,000,000","968,000,000","745,000,000",NaN
AccrualForEnvironmentalLossContingencies,"3,900,000","3,800,000","4,000,000","3,000,000","3,000,000","3,000,000","4,000,000","4,000,000","5,000,000","5,400,000","5,100,000",NaN,NaN,NaN,NaN
AccrualForEnvironmentalLossContingenciesPayments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5,400,000",NaN,NaN,NaN
AccruedIncomeTaxesCurrent,"156,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
balance_df.T.sort_index(axis=1, ascending=False)

,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
Treasury Stock at Cost,"3,099,000,000","2,130,000,000","131,000,000","53,000,000","50,000,000","108,000,000","119,000,000","123,000,000","119,000,000","112,000,000","109,000,000","107,000,000","102,000,000","98,000,000",NaN
Treasury Stock Shares,"33,000,000","25,000,000","6,000,000","5,000,000","5,000,000","12,000,000","14,000,000","14,000,000","12,000,000","10,000,000","9,000,000","9,000,000","8,000,000","8,000,000",NaN
Total Stockholders Equity,"54,750,000,000","7,497,000,000","5,837,000,000","2,827,000,000","1,266,000,000","596,000,000","477,000,000","-350,000,000","187,000,000","544,000,000","538,000,000","1,590,000,000","1,013,000,000","648,000,000","127,000,000"
Total Liabilities and Stockholders Equity,"67,580,000,000","12,419,000,000","8,962,000,000","6,028,000,000","4,556,000,000","3,552,000,000","3,321,000,000","3,084,000,000","3,767,000,000","4,337,000,000","4,000,000,000","4,954,000,000","4,964,000,000","9,078,000,000",NaN
Total Current Liabilities,"6,369,000,000","4,240,000,000","2,417,000,000","2,359,000,000","1,984,000,000","1,513,000,000","1,346,000,000","1,403,000,000","1,440,000,000","1,618,000,000","1,397,000,000","1,774,000,000","1,674,000,000","2,210,000,000",NaN
Total Current Assets,"15,019,000,000","8,583,000,000","6,143,000,000","4,597,000,000","3,540,000,000","2,634,000,000","2,530,000,000","2,320,000,000","2,736,000,000","2,884,000,000","2,265,000,000","3,229,000,000","3,594,000,000","4,275,000,000",NaN
Total Assets,"67,580,000,000","12,419,000,000","8,962,000,000","6,028,000,000","4,556,000,000","3,552,000,000","3,321,000,000","3,084,000,000","3,767,000,000","4,337,000,000","4,000,000,000","4,954,000,000","4,964,000,000","9,078,000,000",NaN
Short Term Debt,0,"312,000,000",0,0,NaN,NaN,0,"230,000,000","172,000,000","55,000,000",NaN,NaN,NaN,NaN,NaN
Retained Earnings,"-131,000,000","-1,451,000,000","-4,605,000,000","-7,095,000,000","-7,436,000,000","-7,775,000,000","-7,742,000,000","-7,306,000,000","-6,646,000,000","-6,243,000,000","-6,160,000,000","-4,977,000,000","-5,468,000,000","-5,939,000,000",NaN
Restricted Cash Noncurrent,NaN,NaN,NaN,0,0,"3,000,000",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
for column in annual_df.columns:
    print(column)

Period End
Filing Date
AccountsPayableCurrent
AccountsPayableRelatedPartiesCurrent
AccountsReceivableGrossCurrent
AccountsReceivableNetCurrent
AccrualForEnvironmentalLossContingencies
AccrualForEnvironmentalLossContingenciesPayments
AccruedIncomeTaxesCurrent
AccruedLiabilitiesCurrent
AccruedMarketingCostsCurrent
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment
AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax
AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax
AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax
AccumulatedOtherComprehensiveIncomeLossNetOfTax
AdditionalPaidInCapitalCommonStock
AdjustmentForAmortization
AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivitiesOther
AdjustmentsToAdditionalPaidInCapitalEquityComponentOfConvertibleDebt
AdjustmentsToAdditionalPaidInCapitalEquityComponentOfConvertibleDebtSub

In [9]:
scorecard = pd.DataFrame()
scorecard["Net Revenue"] = combine_or_add_columns(income_df, ["Net Revenue"])
scorecard["Cost of Revenue"] = combine_or_add_columns(income_df, ["Cost of Revenue"])
scorecard["Gross Profit"] = combine_or_add_columns(income_df, ["Gross Profit"])
scorecard["Gross Margin %"] = scorecard["Gross Profit"] / scorecard["Net Revenue"] * 100
scorecard["SG&A"] = combine_or_add_columns(income_df, ["SG&A"])
scorecard["Operating Income"] = combine_or_add_columns(income_df, ["Operating Income"])
scorecard["Operating Margin %"] = (
    scorecard["Operating Income"] / scorecard["Net Revenue"]
) * 100
scorecard["Interest Expense"] = combine_or_add_columns(income_df, ["Interest Expense"])
scorecard["EBT"] = combine_or_add_columns(
    income_df, ["EBT", "EBT and equity investments"]
)
scorecard["Tax Expense"] = combine_or_add_columns(income_df, ["Taxes"])
scorecard["Tax Rate %"] = combine_or_add_columns(
    income_df, ["Tax Rate Continuing Operations"]
)
scorecard["Tax Rate %"] = scorecard["Tax Rate %"] * 100
scorecard["Net Income"] = combine_or_add_columns(income_df, ["Net Income"])
scorecard["EPS"] = combine_or_add_columns(income_df, ["EPS"])
scorecard["EPS Diluted"] = combine_or_add_columns(income_df, ["EPS Diluted"])
scorecard["EBITDA"] = (
    scorecard["EBT"]
    + scorecard["Interest Expense"].notna()
    + income_df["Depreciation and Amortization"]
)
scorecard["EBITDA Margin %"] = (scorecard["EBITDA"] / scorecard["Net Revenue"]) * 100


scorecard["Cash"] = combine_or_add_columns(
    balance_df, ["Cash", "Cash and Restricted Cash"]
)
scorecard["Marketable Securities"] = combine_or_add_columns(
    balance_df, ["Marketable Securities"]
)
scorecard["Inventory"] = balance_df["Inventory"]
scorecard["Accounts Receivable"] = balance_df["Accounts Receivable"]
scorecard["Other Current Assets"] = combine_or_add_columns(
    balance_df, ["Other Current Assets", "Prepaid Expenses and Other Current Assets"]
)
scorecard["Total Current Assets"] = balance_df["Total Current Assets"]
scorecard["Property, Plant, and Eqipment"] = balance_df[
    "Property, Plant, and Equipment"
]
scorecard["Right of Use Assets"] = combine_or_add_columns(
    balance_df, ["Operating Lease Right of Use Asset"]
)
scorecard["Other Long Term Assets"] = combine_or_add_columns(
    balance_df, ["Other Long Term Assets"]
)
scorecard["Goodwill"] = balance_df["Goodwill Asset"]
scorecard["Intangible Assets"] = combine_or_add_columns(
    balance_df, ["Intangible Assets"]
)
scorecard["Total Assets"] = balance_df["Total Assets"]
scorecard["Accounts Payable"] = combine_or_add_columns(balance_df, ["Accounts Payable"])
scorecard["Accrued Expenses"] = combine_or_add_columns(balance_df, ["Accrued Expenses"])
scorecard["Deferred Revenue"] = combine_or_add_columns(balance_df, ["Deferred Revenue"])
scorecard["Income Tax Payable"] = combine_or_add_columns(
    balance_df, ["Income Taxes Payable"]
)
scorecard["Operating Lease Liability"] = combine_or_add_columns(
    balance_df, ["Operating Lease Liability"]
)
scorecard["Other Current Liabilites"] = combine_or_add_columns(
    balance_df, ["Other Current Liabilities"]
)
scorecard["Total Current Liabilities"] = balance_df["Total Current Liabilities"]
scorecard["Long Term Debt"] = combine_or_add_columns(
    balance_df, ["Long Term Debt", "Long Term Lease Liability"], replace_zero=True
)
scorecard["Other Long Term Liabilities"] = combine_or_add_columns(
    balance_df, ["Other Long Term Liabilities"]
)
scorecard["Total Liabilities"] = combine_or_add_columns(
    balance_df, ["Total Liabilities"]
)
scorecard["Additional Paid In Capital"] = combine_or_add_columns(
    balance_df, ["Additional Paid in Capital"]
)
scorecard["Retained Earnings"] = combine_or_add_columns(
    balance_df, ["Retained Earnings"]
)
scorecard["Total Stockholders Equity"] = combine_or_add_columns(
    balance_df, ["Total Stockholders Equity", "Total Equity(calc)"]
)
scorecard["Total Liabilities and Stockholders Equity"] = combine_or_add_columns(
    balance_df,
    ["Total Liabilities and Stockholders Equity", "Toal Liabilites and Equity(calc)"],
)
scorecard["Shares Outstanding (Diluted)"] = combine_or_add_columns(
    income_df,
    ["Weighted Average Shares Diluted", "Weighted Average Shares Outstanding"],
    replace_zero=True,
)
scorecard["Book Value Per Share"] = (
    scorecard["Total Stockholders Equity"] / scorecard["Shares Outstanding (Diluted)"]
)
scorecard["Operating Cash Flow"] = combine_or_add_columns(
    cash_df, ["Net Cash Provided by Operating Activities"]
)
scorecard["OCF/NI"] = scorecard["Operating Cash Flow"] / scorecard["Net Income"]
scorecard["Depreciation and Amortization"] = combine_or_add_columns(
    cash_df, ["Depreciation and Amortization", "Depreciation and Amortization(calc)"]
)
scorecard["Capex"] = combine_or_add_columns(
    cash_df, ["Purchase of Property, Plant, and Equipment"]
)
scorecard["Capex"] = -scorecard["Capex"]
scorecard["Capex / Depreciation"] = (
    scorecard["Capex"] / scorecard["Depreciation and Amortization"]
)
scorecard["Free Cash Flow"] = scorecard["Operating Cash Flow"] + scorecard["Capex"]
scorecard["Dividend Payment"] = combine_or_add_columns(
    cash_df, ["Payment of Dividends"]
)
if scorecard["Dividend Payment"].isna().all():
    pass
else:
    scorecard["Dividend Payment"] = -scorecard["Dividend Payment"]
scorecard["Stock Repurchase"] = combine_or_add_columns(
    cash_df, ["Repurchases of Common Stock"]
)
scorecard["Stock Repurchase"] = -scorecard["Stock Repurchase"]
scorecard["ROA %"] = (scorecard["Net Income"] / scorecard["Total Assets"]) * 100
scorecard["ROE %"] = (
    scorecard["Net Income"] / scorecard["Total Stockholders Equity"]
) * 100
scorecard["Profit Margin %"] = scorecard["Net Income"] / scorecard["Net Revenue"] * 100
scorecard["Asset Turnover"] = scorecard["Net Revenue"] / scorecard["Total Assets"]
scorecard["Current Ratio"] = (
    scorecard["Total Current Assets"] / scorecard["Total Current Liabilities"]
)
scorecard["Equity Multiplier"] = (
    scorecard["Total Assets"] / scorecard["Total Stockholders Equity"]
)
scorecard["Net Working Capital"] = (
    scorecard["Total Current Assets"] - scorecard["Total Current Liabilities"]
)
scorecard["Debt to Equity Ratio"] = (
    scorecard["Total Liabilities"] / scorecard["Total Stockholders Equity"]
)
scorecard["Debt to Assets Ratio"] = (
    scorecard["Total Liabilities"] / scorecard["Total Assets"]
)
scorecard["Days Sales Outstanding"] = (
    scorecard["Accounts Receivable"] / scorecard["Net Revenue"]
) * 365
scorecard["Days of Inventory on Hand"] = (
    scorecard["Inventory"] / scorecard["Cost of Revenue"]
) * 365
scorecard["Payables Period"] = (
    scorecard["Accounts Payable"] / scorecard["Cost of Revenue"]
) * 365
scorecard["Receivables Turnover"] = (
    scorecard["Net Revenue"] / scorecard["Accounts Receivable"]
)
scorecard["Cash Conversion Cycle"] = (
    scorecard["Days Sales Outstanding"]
    + scorecard["Days of Inventory on Hand"]
    - scorecard["Payables Period"]
)
if scorecard["Dividend Payment"].isna().all():
    scorecard["Dividend"] = np.nan
else:
    scorecard["Dividend"] = (
        -scorecard["Dividend Payment"] / scorecard["Shares Outstanding (Diluted)"]
    )
if "Dividend" in scorecard.columns:
    scorecard["Dividend Payout Ratio %"] = (
        scorecard["Dividend"] / scorecard["EPS"]
    ) * 100
else:
    scorecard["Dividend Payout Ratio %"] = np.nan
scorecard = scorecard[scorecard_order]

In [10]:
scorecard.T

,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
Net Revenue,"23,601,000,000","16,434,000,000","9,763,000,000","6,731,000,000","6,475,000,000","5,291,000,000","4,295,500,000","3,991,000,000","5,506,000,000","5,299,000,000","5,422,000,000","6,568,000,000","6,494,000,000",NaN,NaN
Cost of Revenue,"12,998,000,000","8,505,000,000","5,416,000,000","3,863,000,000","4,028,000,000","3,466,000,000","3,316,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Profit,"10,603,000,000","7,929,000,000","4,347,000,000","2,868,000,000","2,447,000,000","1,787,000,000","1,003,000,000","1,080,000,000","1,839,000,000","1,978,000,000","1,235,000,000","2,940,000,000","2,961,000,000","2,272,000,000","2,320,000,000"
Gross Margin %,44.93,48.25,44.53,42.61,37.79,33.77,23.35,27.06,33.40,37.33,22.78,44.76,45.60,NaN,NaN
SG&A,"2,336,000,000","1,448,000,000","995,000,000","750,000,000","562,000,000","516,000,000","466,000,000","482,000,000","604,000,000","674,000,000","823,000,000","992,000,000","934,000,000","994,000,000","1,304,000,000"
Operating Income,"1,264,000,000","3,648,000,000","1,369,000,000","631,000,000","451,000,000","127,000,000","-373,000,000","-481,000,000","-155,000,000","103,000,000","-1,056,000,000","368,000,000","848,000,000","664,000,000","-1,955,000,000"
Operating Margin %,5.36,22.20,14.02,9.37,6.97,2.40,-8.68,-12.05,-2.82,1.94,-19.48,5.60,13.06,NaN,NaN
Interest Expense,"88,000,000","34,000,000","47,000,000","94,000,000","121,000,000","126,000,000","156,000,000","160,000,000","177,000,000","177,000,000","175,000,000","180,000,000","199,000,000","438,000,000","391,000,000"
EBT,"1,198,000,000","3,675,000,000","1,280,000,000","372,000,000","328,000,000","-15,000,000","-459,000,000","-646,000,000","-398,000,000","-74,000,000","-1,217,000,000","491,000,000","509,000,000","408,000,000","-2,344,000,000"
Tax Expense,"-122,000,000","513,000,000","-1,210,000,000","31,000,000","-9,000,000","18,000,000","39,000,000","14,000,000","5,000,000","9,000,000","-34,000,000","-4,000,000","38,000,000","112,000,000","68,000,000"
